### Notebook for testing and constructing the retrieval of documents from ifixit
##### Features

- _write_searchphrase_ (arg: search_info, returns: search_phrase): rewriting info (str) into search query
- _load_guides_ (arg: search_phrase, returns: guides): loading guides from ifixit using search phrase, stripping words if no guides are found
- _load_ifixit_guides_ (arg: search_info, returns: guides): both functions combined

In [82]:
#development
from dotenv import load_dotenv
import os
from groq import Groq
import warnings
warnings.filterwarnings("ignore")
#production
from langchain_community.document_loaders import IFixitLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain import hub
from langchain.prompts.prompt import PromptTemplate


In [83]:
#initialize test model
load_dotenv()
groq_key = os.getenv('GROQ_API_KEY')
client = Groq(api_key=groq_key)


In [89]:
#function for rewriting info into searchphrase
def write_searchphrase(search_info: str, debug: bool = False):
    """
    Uses the LLM to rewrite input into a structured searchphrase iFixit searches.

    Args:
        search_info (str): The information to be turned into a searchphrase.
    """
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system",
             "content": """Rewrite the following info into a structured searchphrase for iFixit, 
             ensuring it includes the device type, model, and specific repair issue. Do not include any other words or comments."""},
            {"role": "user", "content": search_info},
        ],
        model="llama3-8b-8192",
        temperature=0
    )
    search_phrase = chat_completion.choices[0].message.content
    if debug == True:
        print('Full searchphrase:', search_phrase)
    return search_phrase


#load guides from iFixit
def load_guides(search_phrase: str, debug: bool = False):
    """
    Load a guide from IFixit based on the search phrase.
    If no guide is found, iteratively remove the last word and retry.
    
    Args:
        search_phrase (str): The phrase to search for in IFixit guides.
        
    Returns:
        guides: The loaded guide data or None if no guide is found.
    """
    words = search_phrase.split()
    
    while words:
        query = " ".join(words)
        guides = IFixitLoader.load_suggestions(query, doc_type='guide')
        
        if guides:
            if(debug == True):
                print('Used words:', words)
            return guides  # Return results if found
        
        words.pop()  # Remove the last word and retry

    print('No guides found')
    return None  # Return None if no guide is found


#loading guides from iFixit using optimized searchphrase
def load_ifixit_guides(search_info: str, debug: bool = False):


    search_phrase = write_searchphrase(search_info, debug=debug)
    guides = load_guides(search_phrase, debug=debug)
    return guides



In [90]:
#demonstration of the function
guides = load_ifixit_guides("my iPhone 14 Pro Max is not charging, and the screen is cracked", debug=True) 
print(guides)

Full searchphrase: iPhone 14 Pro Max charging port repair
Used words: ['iPhone', '14', 'Pro', 'Max']
[Document(metadata={'source': 'https://www.ifixit.com/Guide/iPhone+14+Pro+Max+Screen+Replacement/153003', 'title': 'iPhone 14 Pro Max Screen Replacement'}, page_content="# iPhone 14 Pro Max Screen Replacement\nIf your iPhone 14 Pro Max screen is cracked, not responding to touch, or not showing a picture when your phone is powered on, use this guide to restore your iPhone to working order with a new screen, a.k.a. display assembly.\n\nThis guide was created using the ***A2894*** (international) model. If you're fixing a different iPhone 14 model, there may be some visual discrepancies, but the procedure should be the same.\n\n***For your safety, discharge the battery below 25% before disassembling your device.*** This reduces the risk of fire if the battery is accidentally damaged during the repair. If your battery is swollen, [[What to do with a swollen battery|take appropriate precauti